## Overview of Crime Trends in Chicago
#### Cecile Murray

An analysis of crime data from the City of Chicago's open data portal reveals a few key facts about crime trends in Chicago:
* The most common types of crime are theft, battery, and criminal damage. 
* The neighborhoods with the most crime are located on the city's west and south sides. 

In [3]:
import pandas as pd
import geopandas as gpd
import matplotlib as plt

In [5]:
crime17 = pd.read_csv('raw/Crimes-2017.csv')
crime18 = pd.read_csv('raw/Crimes-2018.csv')
comm_areas = gpd.read_file('geo/CHI_comm_areas.geojson')

all_crimes = pd.concat([crime17, crime18])
all_crimes.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,11094370,JA440032,09/21/2017 12:15:00 AM,072XX N CALIFORNIA AVE,1122,DECEPTIVE PRACTICE,COUNTERFEIT CHECK,CURRENCY EXCHANGE,True,False,...,50.0,2.0,10,1156443.0,1947707.0,2017,03/01/2018 03:52:35 PM,42.012293,-87.699714,"(42.012293397, -87.699714109)"
1,11118031,JA470589,10/12/2017 07:14:00 PM,055XX W GRAND AVE,1345,CRIMINAL DAMAGE,TO CITY OF CHICAGO PROPERTY,JAIL / LOCK-UP FACILITY,True,False,...,29.0,19.0,14,1138788.0,1913480.0,2017,03/01/2018 03:52:35 PM,41.918712,-87.765511,"(41.918711651, -87.76551063)"
2,11134189,JA491697,10/30/2017 11:52:00 AM,043XX S TALMAN AVE,4651,OTHER OFFENSE,SEX OFFENDER: FAIL REG NEW ADD,APARTMENT,True,False,...,12.0,58.0,26,1159425.0,1875711.0,2017,03/01/2018 03:52:35 PM,41.814670,-87.690727,"(41.814669784, -87.690726696)"
3,11156462,JA521389,09/29/2017 06:45:00 PM,055XX W BELMONT AVE,1110,DECEPTIVE PRACTICE,BOGUS CHECK,CURRENCY EXCHANGE,True,False,...,30.0,19.0,11,1138653.0,1920720.0,2017,03/01/2018 03:52:35 PM,41.938581,-87.765831,"(41.938581442, -87.765830579)"
4,11164874,JA531910,12/01/2017 06:20:00 AM,022XX W CHICAGO AVE,0265,CRIM SEXUAL ASSAULT,AGGRAVATED: OTHER,STREET,True,False,...,32.0,24.0,02,1161264.0,1905292.0,2017,03/01/2018 03:52:35 PM,41.895805,-87.683159,"(41.895805099, -87.683159355)"


In [6]:
#comm_areas.head()
comm_xwalk = comm_areas[['community', 'area_num_1']]

The table below shows the frequency of each type of crime. Theft, battery, and criminal damage were the most common forms of crime in both 2017 and 2018. While the number of thefts and batteries increased slightly between 2017 and 2018, the number of criminal damage incidents declined by about 1,000.

In [11]:
types17 = crime17[['Primary Type', 'ID']].groupby(['Primary Type']).count().sort_values(by='ID', ascending=False)
types18 = crime18[['Primary Type', 'ID']].groupby(['Primary Type']).count().sort_values(by='ID', ascending=False)
type_summary = types17.join(types18, on='Primary Type', how='outer', lsuffix = '_2017', rsuffix='_2018')
type_summary['Percent Change'] = ((type_summary['ID_2018'] - type_summary['ID_2017']) / type_summary['ID_2017']) * 100
type_summary.columns = ['Number of incidents, 2017', 'Number of incidents, 2018', 'Percent Change']
type_summary

,"Number of incidents, 2017","Number of incidents, 2018",Percent Change
Primary Type,,,
THEFT,64345,65079,1.140726
BATTERY,49214,49781,1.152111
CRIMINAL DAMAGE,29042,27806,-4.255905
ASSAULT,19303,20377,5.563902
DECEPTIVE PRACTICE,19025,18716,-1.624179
OTHER OFFENSE,17227,17125,-0.592094
BURGLARY,13001,11729,-9.783863
ROBBERY,11877,9683,-18.472678
NARCOTICS,11658,12987,11.399897


In [12]:
nhoods17 = crime17[['Community Area', 'ID']].groupby(['Community Area']).count().sort_values(by='ID', ascending=False)
nhoods18 = crime18[['Community Area', 'ID']].groupby(['Community Area']).count().sort_values(by='ID', ascending=False)
nhood_summary = nhoods17.join(nhoods18, on='Community Area', how='outer', lsuffix = '_2017', rsuffix='_2018')
nhood_summary
#pd.merge(nhood_summary, comm_areas[['community', 'area_num_1']], left_on='Community Area', right_on='area_num_1')

,ID_2017,ID_2018
Community Area,,
25,15385,15041
8,12311,13056
32,10624,10856
29,9066,9310
28,9049,9399
43,8677,8648
24,8324,7301
23,8055,8039
71,7527,7370


In [13]:
nhoods_crime = all_crimes[['Community Area', 'Primary Type', 'ID']].groupby(['Community Area', 'Primary Type']).count().sort_values(by='ID', ascending=False).reset_index()
nhoods_crime.columns = ['area_num_1', 'Crime Type', 'Crimes']
nhoods_crime.groupby(['area_num_1', 'Crime Type']).max().reset_index()[['area_num_1', 'Crime Type', 'Crimes']]

,area_num_1,Crime Type,Crimes
0,0.0,ASSAULT,1
1,0.0,BATTERY,1
2,0.0,NARCOTICS,1
3,1.0,ARSON,4
4,1.0,ASSAULT,551
5,1.0,BATTERY,1434
6,1.0,BURGLARY,356
7,1.0,CRIM SEXUAL ASSAULT,57
8,1.0,CRIMINAL DAMAGE,1055
9,1.0,CRIMINAL TRESPASS,335


In [58]:
#nhood_crime_summary.merge(comm_areas[['community', 'area_num_1']], on='area_num_1')

ValueError: You are trying to merge on int64 and object columns. If you wish to proceed you should use pd.concat

Arrest rates vary widely across different types of crime. Nearly all narcotics offenses, concealed carry license violations, and interference with public officer incidents resulted in arrests, while arrests rates were near or below 25 percent for more violent crimes like homicide, assault, battery, and sex offenses. In part, this is because some crimes are only recorded when an arrest occurs; for example, the police can only be aware of a concealed carry license violation if they stop and individual and check them for a concealed weapons. However, the low arrest rates for homicide and other serious violent crimes might still be cause for concern. 

In [14]:
arrests = all_crimes[['Primary Type', 'Arrest', 'ID']].groupby(['Primary Type', 'Arrest']).count().sort_values(by='Primary Type', ascending=False).reset_index()
arrests = arrests.pivot(index='Primary Type', columns='Arrest', values='ID').reset_index()
arrests.columns = ['Primary Type', 'NoArrest', 'Arrest']
arrests['Arrest Rate'] = arrests['Arrest'] / (arrests['Arrest'] + arrests['NoArrest'])
arrests[['Primary Type', 'Arrest Rate']].sort_values(by='Arrest Rate', ascending=False)

,Primary Type,Arrest Rate
17,NARCOTICS,0.999067
4,CONCEALED CARRY LICENSE VIOLATION,0.967890
12,INTERFERENCE WITH PUBLIC OFFICER,0.951903
20,OBSCENITY,0.797688
31,WEAPONS VIOLATION,0.737569
26,PUBLIC PEACE VIOLATION,0.644227
22,OTHER NARCOTIC VIOLATION,0.583333
7,CRIMINAL TRESPASS,0.570064
19,NON-CRIMINAL (SUBJECT SPECIFIED),0.400000
10,HOMICIDE,0.258320
